# Solutions to exercises in the `ray-core` Lessons

First, import everything we'll need and start Ray:

In [1]:
import ray, time, sys
import numpy as np
sys.path.append('../..')
from util.printing import pnd, pd

In [2]:
ray.init(ignore_reinit_error=True)

2020-04-09 16:05:04,209	INFO resource_spec.py:204 -- Starting Ray with 4.49 GiB memory available for workers and up to 2.27 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-09 16:05:04,563	INFO services.py:1146 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:10675',
 'object_store_address': '/tmp/ray/session_2020-04-09_16-05-04_200827_36742/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-09_16-05-04_200827_36742/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-04-09_16-05-04_200827_36742'}

## Exercise 1 in 02-TaskParallelism-Part1

You were asked to convert the regular Python code to Ray code. Here are the three cells appropriately modified.

First, we need the appropriate imports and `ray.init()`.

In [3]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

In [4]:
start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = ray.get(ids)
duration = time.time() - start

In [5]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 

## Exercise 2 in 03-TaskParallelism-Part2

You were asked to use `ray.wait()` with a shorter timeout, `2.5` seconds. First we need to redefine in this notebook the remote functions we used in that lesson:

In [6]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

In [7]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=2.5)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.024)
0: []
10: [ 3.51983479  0.94445356  3.68859664  1.26042292 -0.10789851  2.17373424
  3.00150713 -1.04821399  2.23492285  3.82278505]
Returned   1 completed tasks. (elapsed time:  5.542)
20: [ 1.90913868 -0.66160134  0.79828434 -1.44480661  2.23747017  0.52514003
  1.26983756 -1.66983646  3.123945   -0.32368274 -2.94656064  0.35264673
  2.46192772 -0.42426448  1.36852146 -0.84023993 -0.45143809  2.49641169
 -3.53041769 -0.53494207]
Returned   2 completed tasks. (elapsed time:  8.026)
30: [ 0.24828713  4.48862214  2.04706808 -0.52998407 -1.84999128  1.32390107
 -1.02822613 -1.04734946 -5.28314685 -2.68462863  2.96015075 -1.30092952
 -1.42989006  2.99289148 -0.72597118 -0.28928593  0.59180701 -0.81810978
  2.94739147  2.27211834  3.07881366 -3.37212919  1.03798083  2.3012059
  1.60353497 -2.26633163  1.29095693  0.22765221 -0.62767614 -1.22440643]
40: [-1.4085726   0.60818398 -1.92229757 -2.6291242  -1.09167228  0.99462429
 -2.13531126 

For a timeout of `2.5` seconds, the second call to `ray.wait()` times out before two tasks finish, so it only returns one completed task. Why did the third and last iteration not time out? (That is, they both successfully returned two items.) It's because all the tasks were running in parallel so they had time to finish. If you use a shorter timeout, you'll see more time outs, where zero or one items are returned. 

Try `1.5` seconds, where all but one iteration times out and returns one item. The first iteration returns two items.
Try `0.5` seconds, where you'll get several iterations that time out and return zero items, while all the other iterations time out and return one item.

## Exercise 3 in 03-TaskParallelism-Part2

You were asked to convert the code to use Ray, especially `ray.wait()`.

In [8]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
ids = [slow_square.remote(n) for n in range(4)]
squares = []
waiting_ids = ids
while len(waiting_ids) > 0:
    finished_ids, waiting_ids = ray.wait(waiting_ids)  # We just assign the second list to waiting_ids...
    squares.extend(ray.get(finished_ids))
duration = time.time() - start

In [9]:
assert squares == [0, 1, 4, 9]
assert duration < 4.1, f'duration = {duration}' 